# 크롤링 모델 자동화
1. [크롤링된 이미지 입력]
2. [yolo로 객체탐지]
3. 객체 탐지 되면 카테고리 값을 전달 받아야함
    - 혹은 사용자로부터 카테고리 값을 입력 받음
4. 입력받은 카테고리 값으로 모델 매칭 -> 모델 url가져오기
5. 해당 카테고리의 모델 사용해서 벡터화
    - 처음 구동 시 전체 crops 폴더의 값을 다 가져와서 make_dateframe() 돌려서 npy 저장까지
    - n회차 구동 시 vector_frame에 있는 npy파일 가져오기, 크롤링된 이미지만 벡터화해서 append후 npy 저장
6. 카테고리 값은?

In [1]:
# 모듈 불러오기
import os
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm # 반복문의 진행상황 확인하는 모듈
import time
from scipy.spatial import distance

# 경고끄기 (option)
import warnings
warnings.filterwarnings('ignore')

In [2]:
category = input('category :') # category는 사용자가 선택 하거나 탐지된 값을 입력

category : phone


## 4. 입력받은 카테고리 값으로 모델 매칭 -> 모델 url 가져오기
1. 딕셔너리 형태로 카테고리 : [모델명, 모델 url]
    - 카테고리를 key로, 모델명과 모델url을 value로 만들기
- __모델명 작성법__ : <br>
    a. 첫번째 문자만 대문자 나머지는 소문자 사용<br>
    b. 띄어쓰기는 _로 사용<br>
    c. bit_m 모델같은 경우 R50x1_object 등으로 모델명_파인튜닝대상 으로 작성

In [4]:
def make_dict():
    category_model_dict = dict()

    # 카테고리 별 모델은 변경될 수 있음
    category_model_dict['wallet'] = ['R50x1_object', 'https://tfhub.dev/google/experts/bit/r50x1/in21k/object/1']
    category_model_dict['phone'] = ['Efficientnet_b0', 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2']
    category_model_dict['cap']  = ['Efficientnet_lite0', 'https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2']
    category_model_dict['card'] = ['R50x1_object', 'https://tfhub.dev/google/experts/bit/r50x1/in21k/object/1']
    category_model_dict['bag'] = ['R50x1_object', 'https://tfhub.dev/google/experts/bit/r50x1/in21k/object/1']
    category_model_dict['book'] = ['R101x3', 'https://tfhub.dev/google/bit/m-r101x3/1']
    category_model_dict['shopping'] = ['Mobilenet_v2', 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4']
    category_model_dict['electronics'] = ['R50x1_device', 'https://tfhub.dev/google/experts/bit/r50x1/in21k/device/1']
    category_model_dict['car_key'] = ['Efficientnet_lite0', 'https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2']
    category_model_dict['shoes'] = ['Mobilenet_v2', 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4']
    category_model_dict['document'] = ['R101x3', 'https://tfhub.dev/google/bit/m-r101x3/1']
    category_model_dict['watch'] = ['Mobilenet_v2', 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4']
     # 아래 카테고리들은 테스트 후 다시 모델 삽입 예정임 임시로 넣어 놈
    category_model_dict['cloth'] = ['R50x1_object', 'https://tfhub.dev/google/experts/bit/r50x1/in21k/clothing/1']
    category_model_dict['gloves'] = ['R50x1_object', 'https://tfhub.dev/google/experts/bit/r50x1/in21k/clothing/1']
    category_model_dict['muffler'] = ['R50x1_object', 'https://tfhub.dev/google/experts/bit/r50x1/in21k/clothing/1']
    category_model_dict['necklace'] = ['R50x1_object', 'https://tfhub.dev/google/experts/bit/r50x1/in21k/object/1']
    category_model_dict['ring'] = ['R50x1_object', 'https://tfhub.dev/google/experts/bit/r50x1/in21k/object/1']
    
    return category_model_dict


In [8]:
def select_model(category, image_shape):
    # 카테고리를 입력받아 모델을 생성해서 모델명과 생성된 모델을 리턴함
    model_dict = make_dict()
    model_name = model_dict[category][0]
    model_url = model_dict[category][1]
    
    layer = hub.KerasLayer(model_url, input_shape=IMAGE_SHAPE+(3,))
    model = tf.keras.Sequential([layer])
    model.build([None, 244, 244, 3])
    
    return model_name, model
    

5. 해당 카테고리의 모델 사용해서 벡터화
    - extract 함수 만들기
    - 처음 구동 시 전체 crops 폴더의 값을 다 가져와서 make_dateframe() 돌려서 npy 저장까지
    - n회차 구동 시 vector_frame에 있는 npy파일 가져오기, 크롤링된 이미지만 벡터화해서 append후 npy 저장